In [ ]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

#### Imports and setup

In [ ]:
import numpy as np
import pandas as pd
import h5py
import nibabel.freesurfer.mghformat as mgh

import statsmodels.formula.api as sm
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_rel

In [ ]:
from spacestream.core.constants import SUBJECTS, CORE_ROI_NAMES, ROI_COLORS
from spacestream.core.paths import DATA_PATH, RESULTS_PATH
from spacestream.utils.general_utils import sem
from spacestream.utils.get_utils import get_mapping

In [ ]:
seeds = [0,1,2,3,4]
hemis = ["lh","rh"]
model_types = [
    "MB_RN50_v2",
    "MB_RN50",
    "MB_RN18",
    "TDANN_Supervised",
    "TDANN_Supervised_0.0",
    "TDANN_SimCLR",
    "TDANN_SimCLR_0.0",
]
checkpoint = "checkpoint0"

#### Load and format data

In [ ]:
def correct_for_voxel_noise_ceiling(NC, mapping):

    brain_r = np.sqrt(
        NC[mapping["winning_idx"].astype(int)] / 100
    )  # convert from R^2 to r
    mapping["winning_roi"] = mapping["winning_roi"].astype(np.float32)

    if np.sum(np.isinf(mapping["winning_test_corr"])) > 0:
        mapping["winning_test_corr"][np.isinf(mapping["winning_test_corr"])] = np.nan

    corrected = mapping["winning_test_corr"] / brain_r
    corrected[mapping["winning_test_corr"] == 0] = np.nan

    return corrected

In [ ]:
# read in data
mlong = {
    "model_type": [],
    "hemi": [],
    "subject": [],
    "ROIS": [],
    "result": [],
}


for hidx, hemi in enumerate(hemis):

    for sidx, subj in enumerate(SUBJECTS):

        # get ROI info
        mgh_file = mgh.load(DATA_PATH + "brains/" + hemi + ".ministreams.mgz")
        streams = mgh_file.get_fdata()[:, 0, 0].astype(int)
        # get noise ceiling estimates
        mgh_file = mgh.load(
            DATA_PATH + "brains/NC/subj" + subj + "/" + hemi + ".nc_3trials.mgh"
        )
        NC = mgh_file.get_fdata()[:, 0, 0]
        NC_trim = NC[streams != 0]
        NC_trim[NC_trim == 0] = np.nan  # Set all 0s to nans to avoid dividing by 0

        for mtype in model_types:

            if "TDANN" in mtype:
                supervised = 1 if "Supervised" in mtype else 0
                sw = "0.0" if "0.0" in mtype else "2.5" if "Supervised" in mtype else "0.25"
  
                temp_by_seed = np.zeros((len(seeds),3))
                for midx, seed in enumerate(seeds):
                    mapping = get_mapping(
                        subj_name="subj" + str(subj),
                        spatial_weight=sw,
                        model_seed=seed,
                        supervised=supervised,
                        hemi=hemi,
                        checkpoint=checkpoint,
                    )
                    corrected = correct_for_voxel_noise_ceiling(NC_trim, mapping)
                    for ridx, r in enumerate(CORE_ROI_NAMES):
                        temp_by_seed[midx, ridx] = np.nanmean(
                            corrected[mapping["winning_roi"] == ridx + 5]
                        )

            else:  # not TDANNs
                mapping = get_mapping(
                    subj_name="subj" + str(subj),
                    model_seed=0,
                    hemi=hemi,
                    model_type = "MB18" if "18" in mtype else "MB50_v2" if "50_v2" in mtype else "MB50",
                    checkpoint=checkpoint,
                )

                corrected = correct_for_voxel_noise_ceiling(NC_trim, mapping)

            for ridx, r in enumerate(CORE_ROI_NAMES):
                mlong["model_type"].append(mtype)
                mlong["hemi"].append(hemi)
                mlong["subject"].append(subj)
                mlong["ROIS"].append(r)

                if "MB" not in mtype:
                    mlong["result"].append(
                        np.nanmean(
                            temp_by_seed[:,ridx],
                            axis=0,
                        )
                    )  # mean across seeds

                else:
                    mlong["result"].append(
                        np.nanmean(
                            corrected[
                                (mapping["winning_roi"] == ridx + 5)
                            ]
                        )
                    )

In [ ]:
df = pd.DataFrame(mlong)
df = df.sort_values('ROIS') #just to get the plotting order right later

In [ ]:
d18 = np.mean(df[(df['model_type'] == 'MB_RN18') & (df['ROIS'] == "Dorsal")]['result'])
v18 = np.mean(df[(df['model_type'] == 'MB_RN18') & (df['ROIS'] == "Ventral")]['result'])
l18 = np.mean(df[(df['model_type'] == 'MB_RN18') & (df['ROIS'] == "Lateral")]['result'])

tdannss_d = np.mean(df[(df['model_type'] == 'TDANN_SimCLR') & (df['ROIS'] == "Dorsal")]['result'])
tdannss_v = np.mean(df[(df['model_type'] == 'TDANN_SimCLR') & (df['ROIS'] == "Ventral")]['result'])
tdannss_l = np.mean(df[(df['model_type'] == 'TDANN_SimCLR') & (df['ROIS'] == "Lateral")]['result'])

In [ ]:
print((tdannss_d-d18)/d18)
print((tdannss_v-v18)/v18)
print((tdannss_l-l18)/l18)

In [ ]:
## load subject2subject estimates
s2s_corrected_by_stream= np.zeros((len(SUBJECTS),len(seeds),len(CORE_ROI_NAMES), len(hemis)))

for hidx, hemi in enumerate(hemis):
    
    for sidx, subj in enumerate(SUBJECTS):

        for seedix, seed in enumerate(seeds):

            load_path = (RESULTS_PATH
                            + "mappings/one_to_one/voxel2voxel/target_subj"
                            + subj
                            + "/mode_"
                            + hemi
                            + "_ministreams_HVA_only_radius5_max_iters100_constant_radius_2.0dist_cutoff_constant_dist_cutoff_spherical"
                            + ("_CV_seed" + str(seed))
                            + "_"
                            + checkpoint
                            + "_voxel2voxel_correlation_info.hdf5"
                        )
            with h5py.File(load_path, "r") as f:

                for r, ridx in enumerate(CORE_ROI_NAMES):
                    s2s_corrected_by_stream[sidx,seedix,r,hidx] =  np.nanmean(f['corrected_test_corr'][:][f['winning_roi'][:] == (2-r)+5])
across_seed_corrected_mean = np.mean(np.mean(s2s_corrected_by_stream,axis=-1),axis=1)

In [ ]:
across_seed_corrected_mean

In [ ]:
# Reformat data
rows = []
for i, roi in enumerate(CORE_ROI_NAMES[::-1]):
    for j, subject in enumerate(SUBJECTS):
        rows.append({"subject": subject, "ROI": roi, "result": across_seed_corrected_mean[j, i]})
s2s_reformatted = pd.DataFrame(rows)

In [ ]:
# Save the dataframes for matlab plotting function
# matlab/F02_C.m

s2s_reformatted.to_csv('/oak/stanford/groups/kalanit/biac2/kgs/projects/Dawn/SpaceStreamPaper/Revision/code/new_Fig2c_noiseCeiling_checkpoint0.csv', index=False)
df.to_csv('/oak/stanford/groups/kalanit/biac2/kgs/projects/Dawn/SpaceStreamPaper/Revision/code/new_Fig2c_dataFrame_checkpoint0.csv', index=False)


#### Statistics

In [ ]:
# Cat & SimCLR Cat included in figure for visualization purposes but statistics are
# run on the spatial constraints vs. multiple behaviors comparisons
# i.e. MB v1 RN50, MB v2 RN50, MB v1 RN18, TDANN Supervised, TDANN SimCLR
df = df[~df["model_type"].isin(["TDANN_Supervised_0.0", "TDANN_SimCLR_0.0"])]

In [ ]:
d_sub = across_seed_corrected_mean[:,0]
l_sub = across_seed_corrected_mean[:,1]
v_sub = across_seed_corrected_mean[:,2]

In [ ]:
# set up separate models by ROI
dorsal_df = df[df["ROIS"] == "Dorsal"]
lateral_df = df[df["ROIS"] == "Lateral"]
ventral_df = df[df["ROIS"] == "Ventral"]

In [ ]:
# collapsed across hemispheres, like subject2subject noise ceiling
d_m = dorsal_df[dorsal_df["model_type"]=="MB_RN18"].groupby('subject')['result'].mean().reset_index()['result']
l_m = lateral_df[lateral_df["model_type"]=="MB_RN18"].groupby('subject')['result'].mean().reset_index()['result']
v_m = ventral_df[ventral_df["model_type"]=="MB_RN18"].groupby('subject')['result'].mean().reset_index()['result']

In [ ]:
# bonferroni correction between num possible comparisons against the noise ceiling
# 5 models (MB RN50 v2, MB RN50, MB RN18, TDANN Supervised, TDANN SimCLR) * 3 ROIs (Dorsal, Lateral, Ventral)
correct_by = 5 * 3

In [ ]:
# MB v1 RN18 has the highest functional correspondence, so this tests the smallest difference
# between an MB model and the noise ceiling
print(ttest_rel(d_m,d_sub)[1]*correct_by)
print(ttest_rel(l_m,l_sub)[1]*correct_by)
print(ttest_rel(v_m,v_sub)[1]*correct_by)

In [ ]:
# Dorsal
dorsal_mod = sm.mixedlm('result~model_type', data = dorsal_df, groups=dorsal_df["subject"]).fit()
print(dorsal_mod.summary())

res = pd.concat([dorsal_mod.params,dorsal_mod.pvalues],axis=1)
res.columns=['coefficient','pvalues']
print(res)
res = res[res.index.str.contains('model_type')]
res['corrected_p'] = multipletests(res['pvalues'],method="bonferroni")[1]
print(res)

In [ ]:
# Lateral
lateral_mod = sm.mixedlm('result~model_type', data = lateral_df, groups=lateral_df["subject"]).fit()
print(lateral_mod.summary())

res = pd.concat([lateral_mod.params,lateral_mod.pvalues],axis=1)
res.columns=['coefficient','pvalues']
print(res)

res = res[res.index.str.contains('model_type')]

res['corrected_p'] = multipletests(res['pvalues'],method="bonferroni")[1]
print(res)

In [ ]:
# Ventral
ventral_mod = sm.mixedlm('result~model_type', data = ventral_df, groups=ventral_df["subject"]).fit()
print(ventral_mod.summary())

res = pd.concat([ventral_mod.params,ventral_mod.pvalues],axis=1)
res.columns=['coefficient','pvalues']
print(res)

res = res[res.index.str.contains('model_type')]

res['corrected_p'] = multipletests(res['pvalues'],method="bonferroni")[1]
print(res)

In [ ]:
d_tdann_simclr = dorsal_df[dorsal_df["model_type"]=="TDANN_SimCLR"].groupby('subject')['result'].mean().reset_index()['result']
l_tdann_simclr = lateral_df[lateral_df["model_type"]=="TDANN_SimCLR"].groupby('subject')['result'].mean().reset_index()['result']
v_tdann_simclr = ventral_df[ventral_df["model_type"]=="TDANN_SimCLR"].groupby('subject')['result'].mean().reset_index()['result']

In [ ]:
d_tdann_sup = dorsal_df[dorsal_df["model_type"]=="TDANN_Supervised"].groupby('subject')['result'].mean().reset_index()['result']
l_tdann_sup = lateral_df[lateral_df["model_type"]=="TDANN_Supervised"].groupby('subject')['result'].mean().reset_index()['result']
v_tdann_sup = ventral_df[ventral_df["model_type"]=="TDANN_Supervised"].groupby('subject')['result'].mean().reset_index()['result']

In [ ]:
# Bonferroni correction for number of possible comparisons between models
# (10 ways to choose 2 from 5 models) * 3 streams
correct_by = 10 * 3

In [ ]:
print(ttest_rel(d_tdann_simclr,d_tdann_sup)[1]*correct_by)
print(ttest_rel(l_tdann_simclr,l_tdann_sup)[1]*correct_by)
print(ttest_rel(v_tdann_simclr,v_tdann_sup)[1]*correct_by)